# Overfitting e regolarizzazione
L'overfitting è un problema tipico del machine learning che si manifesta quando un modello si lega troppo ai dati di addestramento e fallisce nel generalizzare su dati nuovi.

L'overffiting è caratterizzato da:
* **Alta variaza**: le previsioni per modelli addestrati con diverse parti del dataset saranno molto diverse tra loro.
* **Basso bias**: l'errore per le predizioni sul set di addestramento è mediamente molto basso

In [17]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [19]:
path = "https://frenzy86.s3.eu-west-2.amazonaws.com/IFAO/boston_houses.csv"
df = pd.read_csv(path)

In [20]:
df.rename(columns={'MEDV':'Price'},inplace=True)

In [22]:
target = 'Price'
X = df.drop(target,axis=1).values
y = df[target].values

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.3,
                                                    random_state=667,
                                                    )

### Creiamo le features polinomiali
Per correggere l'overfitting prima dobbiamo causarlo, un buon modo è aumentare la complessità del nostro modello aumentando il numero di features utilizzando i polinomi.

In [16]:
polyfeats = PolynomialFeatures(degree=2)
X_train_poly = polyfeats.fit_transform(X_train)
X_test_poly = polyfeats.transform(X_test)

print("Numero di esempi nel test: "+str(X_train_poly.shape[0]))
print("Numero di features: "+str(X_train_poly.shape[1]))

Numero di esempi nel test: 354
Numero di features: 105


### Standardizziamo i dati
**NOTA BENE** Per applicare la regolarizzazione è sempre necessario portare i dati sulla stessa scala.

In [6]:
ss = StandardScaler()
X_train_poly = ss.fit_transform(X_train_poly)
X_test_poly = ss.transform(X_test_poly)

Adesso il nostro set di addestramento contiene 354 e 105 features, abbastanza complesso !

### Riconoscere l'overfitting
Evidenziare un problema di overfitting è molto semplice, un modello che ne soffre avrà memorizzato la struttura dei dati di addestramento, piuttosto che imparare da essi, quindi l'errore per le predizioni sul train set sarà molto basso, invece fallirà nel generalizzare, perciò l'errore nel test set sarà decisamente più alto.<br><br>
Quindi per riconoscere l'overfitting è sufficente confrontare questi due valori, scriviamo una funzione che ci permette di farlo in modo da non dover scrivere più volte lo stesso codice.

In [11]:
def overfit_eval(model, X, y):

    """
    model: il nostro modello predittivo già addestrato
    X: una tupla contenente le prorietà del train set e test set (X_train, X_test)
    y: una tupla contenente target del train set e test set (y_train, y_test)
    """

    y_pred_train = model.predict(X[0])
    y_pred_test = model.predict(X[1])

    mse_train = mean_squared_error(y[0], y_pred_train)
    mse_test = mean_squared_error(y[1], y_pred_test)

    r2_train = r2_score(y[0], y_pred_train)
    r2_test = r2_score(y[1], y_pred_test)

    print("Train set:  MSE="+str(mse_train)+" R2="+str(r2_train))
    print("Test set:  MSE="+str(mse_test)+" R2="+str(r2_test))

### Regressione lineare non regolarizzata
Cominciamo eseguendo una regressione lineare (in realtà si tratta di una regressione polinomiale) senza applicare la regolarizzazione.

In [12]:
ll = LinearRegression()
ll.fit(X_train_poly, y_train)

overfit_eval(ll, (X_train_poly, X_test_poly),(y_train, y_test))

Train set:  MSE=7.398251185150577 R2=0.9104023070425126
Test set:  MSE=18.62241862495294 R2=0.790104565399501


Il modello predice in maniera estremamente (o meglio dire eccessivamente) accurata i dati del train set, mentre è molto più scarso sul test set. Siamo di fronte ad un caso di overfitting.

## Regolarizzazione L2: Ridge Regression
La ridge regression è un modello di regressione lineare che applica la **regolarizzazione L2**, la quale consiste nell'aggiungere una penalità per i pesi nella funzione di costo durante la fase di addestramento.<br>
La penalità è data dalla somma dei quadrati dei pesi:
$$\lambda\sum_{j=1}^{M}W_j^2$$<br>
**Lambda** (conosciuto anche come **alpha**) è il **parametro di regolarizzazione** ed è un'altro iperparametro.
Eseguiamo diverse Ridge regression per diversi valori di alpha.

In [13]:
from sklearn.linear_model import Ridge

alphas = [0.0001, 0.001, 0.01, 0.1 ,1 ,10] #alpha corrispone a lambda

for alpha in alphas:
    print("Alpha="+str(alpha))
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_poly, y_train)

    overfit_eval(ridge, (X_train_poly, X_test_poly),(y_train, y_test))

Alpha=0.0001
Train set:  MSE=3.9816045086392857 R2=0.9517801478599095
Test set:  MSE=15.313256561771372 R2=0.8274025138241103
Alpha=0.001
Train set:  MSE=3.9945619159027577 R2=0.9516232251266225
Test set:  MSE=15.26942110175963 R2=0.8278965883648765
Alpha=0.01
Train set:  MSE=4.042479161944748 R2=0.9510429157277126
Test set:  MSE=14.975847339776498 R2=0.8312054921973886
Alpha=0.1
Train set:  MSE=4.242023931413193 R2=0.9486262971865644
Test set:  MSE=15.022596632688073 R2=0.8306785755089305
Alpha=1
Train set:  MSE=4.488510782195564 R2=0.9456411791334245
Test set:  MSE=15.37282788242468 R2=0.8267310785776973
Alpha=10
Train set:  MSE=5.050929457708906 R2=0.9388299186691482
Test set:  MSE=15.752781296493014 R2=0.8224485796939618


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=6.58419e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=6.58419e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


La Ridge regression, applicando la regolarizzazione L2, ci permette di ridurre l'overfitting e portare l'R2 fino ad un valore di 0.791 per alpha uguale a 10.

## Regolarizzazione L1: Lasso
Lasso è un modello di regressione lineare che applica la regolarizzazione L1, questa funziona in egual modo alla L2, con la differenza che il termine di regolarizza sarà dato dalla somma del valore assoluto dei pesi:
$$\lambda\sum_{j=1}^{M}|W_j|$$<br>
e viene sempre applicato alla funzione di costo durante la fase di addestramento

In [14]:
from sklearn.linear_model import Lasso

alphas = [0.0001, 0.001, 0.01, 0.1 ,1 ,10] #alpha corrisponde a lambda

for alpha in alphas:
    print("Alpha="+str(alpha))
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_poly, y_train)

    overfit_eval(lasso, (X_train_poly, X_test_poly),(y_train, y_test))

Alpha=0.0001
Train set:  MSE=4.880684123559174 R2=0.9408917017574555
Test set:  MSE=17.63183414713736 R2=0.8012695576417552
Alpha=0.001
Train set:  MSE=4.921128879302134 R2=0.9404018891360523
Test set:  MSE=17.625025507022954 R2=0.8013462986121218
Alpha=0.01
Train set:  MSE=5.20003931978996 R2=0.9370241000634638
Test set:  MSE=17.56623589027083 R2=0.8020089231834355
Alpha=0.1


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.691e+02, tolerance: 2.923e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.050e+02, tolerance: 2.923e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.013e+03, tolerance: 2.923e

Train set:  MSE=6.347118469933431 R2=0.9231322163033157
Test set:  MSE=18.663480003856904 R2=0.7896417578478119
Alpha=1
Train set:  MSE=7.488937380460963 R2=0.9093040375083271
Test set:  MSE=20.085567531154187 R2=0.7736132448177132
Alpha=10
Train set:  MSE=9.748955381993323 R2=0.8819337314843678
Test set:  MSE=24.24402151192325 R2=0.7267428289421864


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.592e+03, tolerance: 2.923e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.951e+03, tolerance: 2.923e+00
  model = cd_fast.enet_coordinate_descent(


Lasso ci permette di ottenere un modello ancora migliore, con un R2 di 0.803 per Lambda uguale a 0.1.<br>
Da notare che per valori di lambda più grandi il modello peggiora, questo perché l'effetto della regolarizzazione sarà molto pesante e buona parte dei pesi saranno portati a 0.

## L2 ed L1 insieme: ElasticNet
ElasticNet è un modello di regressione lineare che implementa entrambe le tecniche di regolarizzazone L2 ed L1.<br>
Tramite il parametro <span style="font-family: Monaco">l1_ration</span> possiamo controllare l'effetto delle due regolarizzazione
 * **<span style="font-family: Monaco">l1_ration>0.5</span>** l'effetto della regolarizzazione L1 sarà più intenso rispetto alla L2.
 * **<span style="font-family: Monaco">l1_ration<0.5</span>** l'effetto della regolarizzazione L2 sarà più intenso rispetto alla L1.

In [15]:
from sklearn.linear_model import ElasticNet

alphas = [0.0001, 0.001, 0.01, 0.1 ,1 ,10]

for alpha in alphas:
    print("Lambda is: "+str(alpha))
    elastic = ElasticNet(alpha=alpha, l1_ratio=0.5)
    elastic.fit(X_train_poly, y_train)
    overfit_eval(elastic, (X_train_poly, X_test_poly),(y_train, y_test))

Lambda is: 0.0001
Train set:  MSE=4.906781634344437 R2=0.9405756437189037
Test set:  MSE=17.73097388574876 R2=0.8001521421791827
Lambda is: 0.001
Train set:  MSE=5.07265858059044 R2=0.9385667646843142
Test set:  MSE=17.690350407014147 R2=0.8006100141074058
Lambda is: 0.01
Train set:  MSE=5.313812826690839 R2=0.9356462279849297
Test set:  MSE=17.544147970593816 R2=0.8022578786812898
Lambda is: 0.1


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.847e+02, tolerance: 2.923e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.209e+02, tolerance: 2.923e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.015e+03, tolerance: 2.923e

Train set:  MSE=6.181303364346766 R2=0.9251403464068038
Test set:  MSE=18.384697207494217 R2=0.7927839509957676
Lambda is: 1
Train set:  MSE=7.016211587721153 R2=0.9150290581072491
Test set:  MSE=19.23599979779052 R2=0.7831888210201506
Lambda is: 10
Train set:  MSE=9.190466263656552 R2=0.888697402423964
Test set:  MSE=22.753617566227874 R2=0.7435413442270302


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.461e+03, tolerance: 2.923e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.843e+03, tolerance: 2.923e+00
  model = cd_fast.enet_coordinate_descent(


Utilizzando ElasticNet, e quindi entrambe le regolarizzazioni, abbiamo ottenuto un modello ancora migliore, con un R2 di 0.81 sul test set e 0.92 sul test set.<br>
Abbiamo il nostro vincitore!

## Che differenza c'è tra la regolarizzazione L2 ed L1 ?

La differenza principale tra le due tecniche di regolarizzazione viste è la seguente:
* La regolarizzazione L2 riduce la magnitudine dei pesi a valori più bassi.
* La regolarizzazione L1 elimina le feature più deboli portando il loro peso a 0.
Nella pratica la L2 porta quasi sempre a migliori risultati, ma utilizzarle entrambe con ElasticNet è anche un ottima soluzione.